In [12]:
import os, sys
sys.path.append(os.path.dirname('spritelu/'))
import numpy as np

from data_utils import make_env
import gym

In [13]:
class FlatEnvWrapper(gym.ObservationWrapper):
  """Flattens the environment observations so that only the disentangled observation is returned."""
  def observation(self, observation):
    return observation['disentangled'].flatten()
  
config, env = make_env()
_, env2 = make_env()
flatenv = FlatEnvWrapper(env2)

In [8]:
flatenv.reset()

array([ 0.7069915 ,  0.20446672, -0.07703651,  0.04699163,  0.16143799,
        0.8132074 , -0.06893328,  0.0315886 ,  0.4581884 ,  0.69985   ,
       -0.07812575, -0.02240351,  0.25153223,  0.3608165 , -0.05364893,
        0.01943654], dtype=float32)

In [9]:
flatenv.step(flatenv.action_space.sample())

(array([ 0.629955  ,  0.25145835, -0.07703651,  0.04699163,  0.09250471,
         0.844796  , -0.06893328,  0.0315886 ,  0.38006267,  0.6774465 ,
        -0.07812575, -0.02240351,  0.1978833 ,  0.38025305, -0.05364893,
         0.01943654], dtype=float32),
 0.48382326257469815,
 False,
 {'discount': 1.0})

In [17]:
flatenv.action_space.low

array([0., 0.], dtype=float32)

In [5]:
env.reset()

{'image': array([[[  0,   2,   2],
         [  0,   0,   0],
         [  0, 145,  97],
         [  0, 199, 148],
         [  0, 134,  89],
         [  0,   0,   0],
         [  0,   2,   2],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   2,   2],
         [  0,   0,   0],
         [  0, 204, 136],
         [  0, 255, 207],
         [  0, 187, 125],
         [  0,   0,   0],
         [  0,   3,   2],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   1,   1],
         [  0,   0,   0],
         [  0, 126,  84],
         [  0, 172, 128],
         [  0, 115,  77],
         [  0,   0,   0

In [3]:
env.reset()

{'image': array([[[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [ 26,  39,   0],
         [ 48,  65,   0],
         [ 33,  50,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   3,   2],
         [  0,   4,   3],
         [  0,   2,   1],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [106, 160,   0],
         [196, 255,   0],
         [137, 206,   0],
         [  0,   0,   0],
         [  1,   1,   0

In [15]:
X = list(np.random.randn(100, 10))

In [16]:
pairwise_distance(X).shape

(4950,)

In [17]:
squareform(pairwise_distance(X))

array([[0.        , 3.43100307, 3.90939724, ..., 4.13918526, 4.89667661,
        4.16458682],
       [3.43100307, 0.        , 3.33602389, ..., 3.95652384, 5.84240365,
        4.09021974],
       [3.90939724, 3.33602389, 0.        , ..., 3.00628503, 5.57906207,
        3.91229246],
       ...,
       [4.13918526, 3.95652384, 3.00628503, ..., 0.        , 5.08458981,
        3.52459837],
       [4.89667661, 5.84240365, 5.57906207, ..., 5.08458981, 0.        ,
        5.7211318 ],
       [4.16458682, 4.09021974, 3.91229246, ..., 3.52459837, 5.7211318 ,
        0.        ]])

In [14]:
from spriteworld import tasks
from scipy.spatial.distance import pdist as pairwise_distance, squareform
class PairwiseDistanceSprites(tasks.AbstractTask):
  """Task is to min/max a function of pairwise distance between all the sprites"""
  
  def __init__(self, mode='max', fn=np.mean):
    if mode == 'max':
      self.coef = 1.
    elif mode == 'min':
      self.coef = -1.
    else:
      raise NotImplementedError
      
    self.fn = fn
    
  def reward(self, sprites):
    """Computes reward from list of sprites"""
    poses = [s.position for s in sprites]
    return self.coef * self(pairwise_distance(poses))
    
    
  def reward_of_vector_repr(self, state_vector):
    """Computes reward on a 'VectorizedPositionsAndVelocities' format"""
    poses = state_vector.reshape(-1, 4)[:,:2]
    return self.coef * self(pairwise_distance(poses))
    
  def success(self, sprites):
    return False # never terminates

SyntaxError: unexpected EOF while parsing (<ipython-input-14-2b9a05b06c62>, line 1)